In [1]:
import django
import sys, os
import pandas as pd
import matplotlib.pyplot as plt

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
import parliament.models as pm

from django.db.models import Count, Sum
import random
p = Project.objects.get(pk=134)
p

<Project: Coal-discourse>

In [9]:
par_search_pdf = pm.Search.objects.get(
    title="Kohle pdf utterance all",
    creator=User.objects.get(username="leey")
)

uts = par_search_pdf.utterance_set.all()

uts.count()

8594

In [10]:
q, created = Query.objects.get_or_create(
    title="Kohle tei utterance",
    text="kohle",
    project=p,
    creator=User.objects.get(username="galm")
)


users = User.objects.filter(username__in=["galm","minj","leia"])
print(users)

for u in users:
    q.users.add(u)

tag, created = Tag.objects.get_or_create(
    query=q,
    title="Kohle tei utterance",
    text="kohle",
    document_linked=False,
    utterance_linked=True
)



<QuerySet [<User: minj>, <User: galm>, <User: leia>]>


In [12]:
sample_size = 200

ut_ids = list(uts.values_list('pk',flat=True))
ut_sample = uts.filter(id__in=random.sample(ut_ids,sample_size*users.count()))

for i,ut in enumerate(ut_sample):
    u = users[i%users.count()]
    do = DocOwnership.objects.filter(tag=tag)[i]
    if do.relevant==0:
        do.utterance = ut
        do.save()
    #do, created = DocOwnership.objects.get_or_create(
    #    utterance=ut,
    #    document_linked=False,
    #    utterance_linked=True,
    #    user=u,
    #    query=q,
    #    tag=tag
    #)